In [1]:
from Fed_Envs import standard
import gym 
import numpy as np 

In [2]:
env = standard.MECStandard()

/home/manoli/Federation-RL-Simulator/.venv/lib/python3.7/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [3]:
print('Example State Space', env.observation_space.sample())
states = env.observation_space.shape
action = env.action_space.n

print('States and Actions', states, action)

Example State Space [74.082634]
States and Actions (1,) 8


In [7]:
episodes = 15
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-11.2
Episode:2 Score:-9.4
Episode:3 Score:-12.6
Episode:4 Score:-25.2
Episode:5 Score:-12.6
Episode:6 Score:-19.2
Episode:7 Score:-9.4
Episode:8 Score:-12.4
Episode:9 Score:-9.4
Episode:10 Score:-11.2
Episode:11 Score:-19.2
Episode:12 Score:-19.2
Episode:13 Score:-25.2
Episode:14 Score:-18.2
Episode:15 Score:-11.2


In [8]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
from rl.policy import EpsGreedyQPolicy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
#from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import pandas as pd
import tensorflow.keras.callbacks 
from keras_tqdm import TQDMNotebookCallback
from tqdm.keras import TqdmCallback


states = env.observation_space.shape
action = env.action_space.n


def build_model(state,actions):
    model = Sequential()
    model.add(Dense(24,activation='relu', input_shape=states))
    model.add(Dense(24,activation='relu'))
    model.add(Dense(action,activation='linear'))
    return model

2025-02-07 11:37:35.386422: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.8/lib64:
2025-02-07 11:37:35.386456: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.8/lib64:
2025-02-07 11:37:35.386458: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
Using TensorFlow backend.
/home/manoli/Federation-RL-Simulator/.venv/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See

In [9]:
model = build_model(states,action)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [10]:
def build_agent(model,action):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000,window_length=1)
    dqn = DQNAgent(model=model,memory=memory,policy=policy,nb_actions=action, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [11]:
dqn1 = build_agent(model,action)
dqn1.compile(tf.keras.optimizers.Adam(learning_rate=.0001), metrics=['mae'])
holder = dqn1.fit(env,nb_steps=10000,visualize=False);
df1 = pd.DataFrame(holder.history)

Training for 10000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 3:21 - reward: -12.4000

2025-02-07 11:37:59.216414: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2025-02-07 11:37:59.239697: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-02-07 11:37:59.242656: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3050 OEM computeCapability: 8.6
coreClock: 1.755GHz coreCount: 18 deviceMemorySize: 7.66GiB deviceMemoryBandwidth: 208.65GiB/s
2025-02-07 11:37:59.242718: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.8/lib64:
2025-02-07 11:37:59.242744: W tensorflow/stream_executor/platform/defa

10000/10000 [==============================] - 31s 3ms/step - reward: -10.6155
done, took 31.157 seconds
